In [ ]:
%pylab inline
import seaborn as sns
sns.set_palette("colorblind")

In [ ]:
from io import StringIO
import pandas as pd

In [ ]:
def get_cv_scores():
    data = !jq -f scripts/jq-ridge.txt cv/ridge/cv.json
    
    return pd.read_json('\n'.join(data))#.set_index(["score", "alpha"])

In [ ]:
df= get_cv_scores()
df.head()

In [ ]:
fg = sns.FacetGrid(df, hue="score", aspect=1.5, size=4, legend_out=False)
fg.map(plt.semilogx, "alpha", "test_error")
fg.map(plt.semilogx, "alpha", "train_error", ls='--')

fg.add_legend()
plt.ylabel(r'$R^2$')
plt.title("Cross Validation Performance for Ridge Regression\n"
          "training(--), test(-)")

The best value of the regularization parameter is between .1-.15.